In [ ]:
!pip install gradio
!pip install tensorflow

In [47]:
#Get cvs
from google.colab import drive
drive.mount('/content/drive')

# path for cvs
csv_path = '/content/drive/MyDrive/Colab Notebooks/nytcrosswords.csv'
import pandas as pd
df = pd.read_csv(csv_path, encoding='latin1')
!ls "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'11 19 Class.ipynb'	    Final_Blah_Working_Decoder.ipynb
 answer_vectorizer.pkl	    Final_Crossword.ipynb
'Binomial Code.ipynb'	   'Final_Crossword_UI_FIXED_SAFE (1).ipynb'
 CACA.ipynb		   'Final Draft.ipynb'
'CH10 document.ipynb.txt'  'HW 5'
'Class 11 14 DV.ipynb'	    nyt_crossword_colab_drive_upload.ipynb
 clue_vectorizer.pkl	    nyt_crossword_generator_fixed.ipynb
 crossword_model.h5	    nytcrosswords.csv
 crossword_model.keras	   'NYT Solver.ipynb'
 Final_Blah_FIXED.ipynb    'Try 2.ipynb'
'Final Blah.ipynb'


In [48]:
# Preprocess and split
df = df.dropna(subset=["Word"])
df["input_text"] = df["Clue"].str.lower().str.strip()
df["target_text"] = df["Word"].str.upper().str.strip()
df = df[["input_text", "target_text"]]

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [49]:
# Tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

clue_vectorizer = TextVectorization(output_mode='int', output_sequence_length=20)
clue_vectorizer.adapt(train_df['input_text'])

answer_vectorizer = TextVectorization(output_mode='int', output_sequence_length=16, split='character')
answer_vectorizer.adapt(train_df['target_text'])

In [69]:
# Encoder-decoder LSTM model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

clue_vocab_size = len(clue_vectorizer.get_vocabulary())
answer_vocab_size = len(answer_vectorizer.get_vocabulary())

encoder_inputs = Input(shape=(None,), name='encoder_input')
x = Embedding(clue_vocab_size, 128)(encoder_inputs)
_, state_h, state_c = LSTM(256, return_state=True)(x)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,), name='decoder_input')
y = Embedding(answer_vocab_size, 128)(decoder_inputs)
y, _, _ = LSTM(256, return_sequences=True, return_state=True)(y, initial_state=encoder_states)
decoder_outputs = Dense(answer_vocab_size, activation='softmax')(y)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [51]:
# Format data
def format_dataset(df):
    clues = tf.convert_to_tensor(df['input_text'].values)
    answers = tf.convert_to_tensor(df['target_text'].values)
    encoder_input = clue_vectorizer(clues)
    decoder_input = answer_vectorizer(answers)[:, :-1]
    decoder_target = answer_vectorizer(answers)[:, 1:]
    return tf.data.Dataset.from_tensor_slices(((encoder_input, decoder_input), decoder_target)).shuffle(1024).batch(64).prefetch(tf.data.AUTOTUNE)

train_ds = format_dataset(train_df)
val_ds = format_dataset(val_df)

In [53]:
# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10) #Run '1' for test (Makes generator really really bad)

10991/10991 ━━━━━━━━━━━━━━━━━━━━ 4202s 382ms/step - accuracy: 0.7918 - loss: 0.6654 - val_accuracy: 0.8276 - val_loss: 0.5523


In [54]:
model.save('/content/drive/MyDrive/Colab Notebooks/crossword_model.keras')

In [55]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/clue_vectorizer.pkl', 'wb') as f:
    pickle.dump(clue_vectorizer, f)

with open('/content/drive/MyDrive/Colab Notebooks/answer_vectorizer.pkl', 'wb') as f:
    pickle.dump(answer_vectorizer, f)


In [67]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/crossword_model.keras')

# Load vectorizers
with open('/content/drive/MyDrive/Colab Notebooks/clue_vectorizer.pkl', 'rb') as f:
    clue_vectorizer = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/answer_vectorizer.pkl', 'rb') as f:
    answer_vectorizer = pickle.load(f)


In [68]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model

global decoder_lstm_infer
decoder_lstm = decoder_lstm_infer

# Lookup tables
answer_vocab = answer_vectorizer.get_vocabulary()
answer_index_lookup = {i: c for i, c in enumerate(answer_vocab)}
answer_token_lookup = {c: i for i, c in enumerate(answer_vocab)}



encoder_inputs = Input(shape=(None,), dtype='int32')
embedding_layer = model.get_layer('embedding_15')
lstm_layer = model.get_layer('lstm_15')  # Encoder LSTM
x = embedding_layer(encoder_inputs)
lstm_output = lstm_layer(x)
if isinstance(lstm_output, (list, tuple)):
    lstm_output = lstm_output[0]
state_h = state_c = lstm_output
encoder_model = Model(encoder_inputs, [state_h, state_c])

# Create an inference-time LSTM layer and copy weights
decoder_lstm_trained = model.get_layer("lstm_15")

# Create a new layer
decoder_lstm_infer = LSTM(256, return_sequences=True, return_state=True, name="lstm_3_infer")
_ = decoder_lstm_infer(tf.zeros((1, 1, 128)))  # batch, sequence, embedding dim
decoder_lstm_infer.set_weights(decoder_lstm_trained.get_weights())


def decode_sequence(clue_text):
    try:
        # Vectorize the clue
        clue_vec = clue_vectorizer([clue_text])

        # Run the encoder
        encoder_inputs = Input(shape=(None,), dtype='int32')
        embedding_layer = model.get_layer('embedding_14')   #Mathc with mcurrent modle
        lstm_layer = model.get_layer('lstm_14')
        x = embedding_layer(encoder_inputs)
        lstm_output = lstm_layer(x)
        if isinstance(lstm_output, (list, tuple)):
            lstm_output = lstm_output[0]
        state_h = state_c = lstm_output
        encoder_model = tf.keras.Model(encoder_inputs, [state_h, state_c])

        state_h, state_c = encoder_model.predict(clue_vec)

        # Lookup vocab
        answer_vocab = answer_vectorizer.get_vocabulary()
        answer_index_lookup = {i: str(c) for i, c in enumerate(answer_vocab)}
        answer_token_lookup = {str(c): i for i, c in enumerate(answer_vocab)}

        # Start decoding
        start_token_id = answer_token_lookup.get('e', 2)
        target_seq = tf.constant([[start_token_id]])

        decoded_chars = []
        decoder_embedding = model.get_layer('embedding_15')
        decoder_lstm = model.get_layer('lstm_15')
        decoder_dense = model.get_layer('dense_7')

        for _ in range(16):
            embedded = decoder_embedding(target_seq)
            output, h, c = decoder_lstm(embedded, initial_state=[state_h, state_c])
            logits = decoder_dense(output)
            token_index = tf.argmax(logits[0, -1, :]).numpy()

            #prevent out-of-bounds index
            if token_index >= len(answer_vocab):
                print("Token out of range:", token_index)
                token_index = 1  # [UNK]

            token_char = answer_index_lookup.get(token_index, '')
            if token_char == '' or token_char == '[END]':
                break

            decoded_chars.append(token_char)
            target_seq = tf.constant([[token_index]])
            state_h, state_c = h, c

        return ''.join(decoded_chars)

    except Exception as e:
        print("Error in decode_sequence:", e)
        return f"Error: {str(e)}"


#Gradio UI
interface = gr.Interface(
    fn=decode_sequence,
    inputs=gr.Textbox(label='Crossword Clue'),
    outputs=gr.Textbox(label='Predicted Answer'),
    title='Crossword Clue Answer Generator'
)
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac86d39208e965575e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
